Descargamos el dataset en un pandas dataframe

In [18]:
import pandas as pd
import os
import time
file = "Online Retail.xlsx"
url = "https://github.com/Alf-caput/LAB02_ReglasYPatronesSecuenciales/raw/main/Online%20Retail.xlsx"
if os.path.exists(file):
    target = file
else:
    target = url
df = pd.read_excel(target)

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


Con **.info** podemos ver el número de filas, columnas y los tipos almacenados.

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   541909 non-null  int64         
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 33.1+ MB


Observamos 541909 entradas y 8 columnas.  
Buscando documentación en kaggle:  
Se trata de un dataset que contiene las transacciones que han ocurrido entre 01/12/2009 y 09/12/2011 en UK de una compañía que vende principalmente regalos de ocasión, adicionalmente se especifica que muchos de los compradores son mayoristas.  
Columnas del dataset:
- **InvoiceNo:**  
    - Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
- **StockCode:**
    - Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
- **Description:**
    - Product (item) name. Nominal.
- **Quantity:** 
    - The quantities of each product (item) per transaction. Numeric.
- **InvoiceDate:**
    - Invoice date and time. Numeric. The day and time when a transaction was generated.
- **UnitPrice**:
    - Unit price. Numeric. Product price per unit in sterling (Â£).
- **CustomerID:**
    - Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
- **Country:**
    - Country name. Nominal. The name of the country where a customer resides.

Viendo los tipos que devuelve **.info** vemos que pandas nos ha guardado Quantity, UnitPrice y CustomerID como tipos numéricos, mientras que el resto excepto InvoiceDate (tipo fecha) son de tipo object, que es el tipo por defecto que usa pandas para strings.

Con **.isna** podemos ver cuantos valores na tiene en total y por columna el dataframe.  
(De **.info** también se puede deducir pero es menos intuitivo)

In [20]:
print(f'Número total de NA en el dataframe: {(col_na:=df.isna().sum()).sum()}')
pd.DataFrame({'Valores NA': col_na})

Número total de NA en el dataframe: 1454


,Valores NA
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,0
Country,0


Encontramos pocos valores NA, que podríamos completar si conocemos su StockCode, no obstante estudiamos un poco más el dataset antes de tomar una acción en este aspecto.  
Con **.describe** podemos ver el rango de valores que pueden tomar diferentes columnas (omitimos la columna "count" y transponemos por legibilidad).

In [21]:
(
    df
    .describe()
    .drop('count')
    .T
)

,mean,min,25%,50%,75%,max,std
Quantity,9.55225,-80995.0,1.0,3.0,10.0,80995.0,218.081158
InvoiceDate,2011-07-04 13:34:57.156386048,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
UnitPrice,4.611114,-11062.06,1.25,2.08,4.13,38970.0,96.759853
CustomerID,15287.518434,12346.0,14367.0,15287.0,16255.0,18287.0,1484.746041


Vemos que existen valores que no nos interesan para el problema que queremos resolver, existen cantidades y precio de unidades negativas ("Quantity", "UnitPrice") que pueden deberse a devoluciones u otro tipo de gestión en el producto.  

In [22]:
df_negs = (
    df
    .loc[(df.loc[:, 'Quantity'] <= 0) | (df.loc[:, 'UnitPrice'] <= 0), :]
)
print(f"Entradas con cantidad o precio unidad negativas (o cero): {df_negs.shape[0]}")

negs_ratio = round(len(df_negs)/len(df), 2)
print(f"Ratio: {negs_ratio}")

df_negs.head()

Entradas con cantidad o precio unidad negativas (o cero): 11805
Ratio: 0.02


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom


Tenemos más de 11_000 entradas, que suponen un 2% del dataset.  
Como no nos interesan podemos eliminar esas entradas.  

In [23]:
df_std = df.drop(df_negs.index)
df_std.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [80]:
import numpy as np
df_invoice = df_std.groupby(['InvoiceNo', 'CustomerID', 'InvoiceDate'])['StockCode'].apply(lambda x: np.array(x)).reset_index()
df_invoice.sort_values(by='CustomerID')
df_invoice.head()

,InvoiceNo,CustomerID,InvoiceDate,StockCode
0,536365,17850,2010-12-01 08:26:00,"[85123A, 71053, 84406B, 84029G, 84029E, 22752,..."
1,536366,17850,2010-12-01 08:28:00,"[22633, 22632]"
2,536367,13047,2010-12-01 08:34:00,"[84879, 22745, 22748, 22749, 22310, 84969, 226..."
3,536368,13047,2010-12-01 08:34:00,"[22960, 22913, 22912, 22914]"
4,536369,13047,2010-12-01 08:35:00,[21756]


In [81]:
df_testing = df_invoice.loc[df_invoice.loc[:, 'CustomerID'] == 13047, :]
df_testing

,InvoiceNo,CustomerID,InvoiceDate,StockCode
2,536367,13047,2010-12-01 08:34:00,"[84879, 22745, 22748, 22749, 22310, 84969, 226..."
3,536368,13047,2010-12-01 08:34:00,"[22960, 22913, 22912, 22914]"
4,536369,13047,2010-12-01 08:35:00,[21756]
2800,543113,13047,2011-02-03 13:06:00,"[16161P, 22766, 22708, 23231, 85015, 85016, 22..."
5126,548331,13047,2011-03-30 13:28:00,"[22960, 22722, 22720, 20972, 22274, 84969, 226..."
6602,551846,13047,2011-05-04 14:11:00,"[23152, 23182, 23161, 23160, 23163, 23162, 231..."
9123,557299,13047,2011-06-19 13:56:00,"[23092, 23112, 23110, 23111, 23118, 23093, 230..."
11388,562437,13047,2011-08-04 17:58:00,"[22623, 16161P, 21754, 21755, 21901, 21900, 48..."
15263,571185,13047,2011-10-14 11:15:00,"[22739, 23435, 22960, 22722, 22720, 20972, 226..."
16998,575043,13047,2011-11-08 12:06:00,[M]


In [82]:
max_distance = pd.Timedelta(minutes=5)
df_testing.loc[:, 'InvoiceGroup'] = (
    df_testing
    .sort_values('InvoiceDate')
    .groupby('CustomerID')['InvoiceDate']
    .diff()
    .gt(max_distance)
    .cumsum()
)
df_testing

C:\Users\alfre\AppData\Local\Temp\ipykernel_18644\425054327.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_testing.loc[:, 'InvoiceGroup'] = (


,InvoiceNo,CustomerID,InvoiceDate,StockCode,InvoiceGroup
2,536367,13047,2010-12-01 08:34:00,"[84879, 22745, 22748, 22749, 22310, 84969, 226...",0
3,536368,13047,2010-12-01 08:34:00,"[22960, 22913, 22912, 22914]",0
4,536369,13047,2010-12-01 08:35:00,[21756],0
2800,543113,13047,2011-02-03 13:06:00,"[16161P, 22766, 22708, 23231, 85015, 85016, 22...",1
5126,548331,13047,2011-03-30 13:28:00,"[22960, 22722, 22720, 20972, 22274, 84969, 226...",2
6602,551846,13047,2011-05-04 14:11:00,"[23152, 23182, 23161, 23160, 23163, 23162, 231...",3
9123,557299,13047,2011-06-19 13:56:00,"[23092, 23112, 23110, 23111, 23118, 23093, 230...",4
11388,562437,13047,2011-08-04 17:58:00,"[22623, 16161P, 21754, 21755, 21901, 21900, 48...",5
15263,571185,13047,2011-10-14 11:15:00,"[22739, 23435, 22960, 22722, 22720, 20972, 226...",6
16998,575043,13047,2011-11-08 12:06:00,[M],7


In [76]:
nuevo_df = (
    df_testing
    # .loc[:, ['StockCode', 'InvoiceGroup']]
    .groupby('InvoiceGroup')
    .agg({
        'StockCode': lambda x: np.concatenate(x.values),
        'InvoiceDate': 'max',
        }))
nuevo_df

,StockCode,InvoiceDate
InvoiceGroup,,
0,"[84879, 22745, 22748, 22749, 22310, 84969, 226...",2010-12-01 08:35:00
1,"[16161P, 22766, 22708, 23231, 85015, 85016, 22...",2011-02-03 13:06:00
2,"[22960, 22722, 22720, 20972, 22274, 84969, 226...",2011-03-30 13:28:00
3,"[23152, 23182, 23161, 23160, 23163, 23162, 231...",2011-05-04 14:11:00
4,"[23092, 23112, 23110, 23111, 23118, 23093, 230...",2011-06-19 13:56:00
5,"[22623, 16161P, 21754, 21755, 21901, 21900, 48...",2011-08-04 17:58:00
6,"[22739, 23435, 22960, 22722, 22720, 20972, 226...",2011-10-14 11:15:00
7,[M],2011-11-08 12:06:00


In [88]:
nuevo_df = (
    df_testing.loc[:, ['InvoiceGroup', 'StockCode', 'InvoiceDate', 'InvoiceNo']]
    .groupby('InvoiceGroup')
    .apply(lambda x: pd.Series({
        'StockCode': np.concatenate(x['StockCode'].values),
        'InvoiceDate': x.loc[x['InvoiceDate'].idxmax(), 'InvoiceDate'],
        'InvoiceNo': x.loc[x['InvoiceDate'].idxmax(), 'InvoiceNo']
})).reset_index())
nuevo_df


C:\Users\alfre\AppData\Local\Temp\ipykernel_18644\3252556709.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


,InvoiceGroup,StockCode,InvoiceDate,InvoiceNo
0,0,"[84879, 22745, 22748, 22749, 22310, 84969, 226...",2010-12-01 08:35:00,536369
1,1,"[16161P, 22766, 22708, 23231, 85015, 85016, 22...",2011-02-03 13:06:00,543113
2,2,"[22960, 22722, 22720, 20972, 22274, 84969, 226...",2011-03-30 13:28:00,548331
3,3,"[23152, 23182, 23161, 23160, 23163, 23162, 231...",2011-05-04 14:11:00,551846
4,4,"[23092, 23112, 23110, 23111, 23118, 23093, 230...",2011-06-19 13:56:00,557299
5,5,"[22623, 16161P, 21754, 21755, 21901, 21900, 48...",2011-08-04 17:58:00,562437
6,6,"[22739, 23435, 22960, 22722, 22720, 20972, 226...",2011-10-14 11:15:00,571185
7,7,[M],2011-11-08 12:06:00,575043


Adicionalmente vamos a meter en la factura más reciente las facturas de un mismo cliente que disten menos de 10 minutos.  
Nota: Elegimos de manera arbitraria 10 minutos, pensamos que es improbable que para un comercio de estas características se den transacciones cada menos de 10 minutos para un mismo cliente.  

In [25]:
from itertools import combinations

def check_time_diff(dates):
    for date1, date2 in combinations(dates, 2):
        if abs((date1 - date2).total_seconds()) / 60 < 10:
            return True
    return False

df_invoice_filtered_10min = (
    df_invoice
    .groupby(['CustomerID'])
    .filter(lambda x: check_time_diff(x['InvoiceDate']))
)
print(df_invoice_filtered_10min.sort_values(by='CustomerID').head())

      InvoiceNo  CustomerID         InvoiceDate  \
12031    563852       12395 2011-08-19 15:18:00   
9678     558623       12395 2011-06-30 16:52:00   
18193    577551       12395 2011-11-20 15:21:00   
332      537026       12395 2010-12-03 16:35:00   
7120     552884       12395 2011-05-12 10:15:00   

                                               StockCode  
12031   [85066, 84816, 22457, 22326, 22352, 22329, POST]  
9678   [22326, 22661, 21559, 21561, 23307, 22726, 227...  
18193  [23487, 23407, 23389, 23390, 21559, 84375, 226...  
332    [84375, 21217, 21212, 21977, 22417, 21975, 219...  
7120   [22727, 22728, 22729, 22730, 22666, 84380, 229...  


In [26]:
(
    df_invoice
    .loc[df_invoice.loc[:, 'CustomerID'] == 13047]
)

,InvoiceNo,CustomerID,InvoiceDate,StockCode
2,536367,13047,2010-12-01 08:34:00,"[84879, 22745, 22748, 22749, 22310, 84969, 226..."
3,536368,13047,2010-12-01 08:34:00,"[22960, 22913, 22912, 22914]"
4,536369,13047,2010-12-01 08:35:00,[21756]
2800,543113,13047,2011-02-03 13:06:00,"[16161P, 22766, 22708, 23231, 85015, 85016, 22..."
5126,548331,13047,2011-03-30 13:28:00,"[22960, 22722, 22720, 20972, 22274, 84969, 226..."
6602,551846,13047,2011-05-04 14:11:00,"[23152, 23182, 23161, 23160, 23163, 23162, 231..."
9123,557299,13047,2011-06-19 13:56:00,"[23092, 23112, 23110, 23111, 23118, 23093, 230..."
11388,562437,13047,2011-08-04 17:58:00,"[22623, 16161P, 21754, 21755, 21901, 21900, 48..."
15263,571185,13047,2011-10-14 11:15:00,"[22739, 23435, 22960, 22722, 22720, 20972, 226..."
16998,575043,13047,2011-11-08 12:06:00,[M]


In [27]:
(
    df_invoice_filtered_10min
    .loc[df_invoice_filtered_10min.loc[:, 'CustomerID'] == 13047]
)

,InvoiceNo,CustomerID,InvoiceDate,StockCode
2,536367,13047,2010-12-01 08:34:00,"[84879, 22745, 22748, 22749, 22310, 84969, 226..."
3,536368,13047,2010-12-01 08:34:00,"[22960, 22913, 22912, 22914]"
4,536369,13047,2010-12-01 08:35:00,[21756]
2800,543113,13047,2011-02-03 13:06:00,"[16161P, 22766, 22708, 23231, 85015, 85016, 22..."
5126,548331,13047,2011-03-30 13:28:00,"[22960, 22722, 22720, 20972, 22274, 84969, 226..."
6602,551846,13047,2011-05-04 14:11:00,"[23152, 23182, 23161, 23160, 23163, 23162, 231..."
9123,557299,13047,2011-06-19 13:56:00,"[23092, 23112, 23110, 23111, 23118, 23093, 230..."
11388,562437,13047,2011-08-04 17:58:00,"[22623, 16161P, 21754, 21755, 21901, 21900, 48..."
15263,571185,13047,2011-10-14 11:15:00,"[22739, 23435, 22960, 22722, 22720, 20972, 226..."
16998,575043,13047,2011-11-08 12:06:00,[M]


In [28]:
import pandas as pd

# Suponiendo que tienes un DataFrame llamado df con las columnas 'id', 'valor' y 'lista'
# Aquí tienes un ejemplo de cómo se vería:
data = {'id': [1, 1, 2, 2, 3],
        'valor': [10, 4, 8, 2, 6],
        'lista': [['a', 'b'], ['c', 'd'], ['e', 'f'], ['g', 'h'], ['i', 'j']]}
df = pd.DataFrame(data)
print(df)
# Definir una función para combinar listas y mantener el id del valor más grande
def combinar_filas(group):
    max_id = group.loc[group['valor'].idxmax(), 'id']
    combined_list = sum(group['lista'], [])
    return pd.Series({'id': max_id, 'lista': combined_list})

# Aplicar la función a las filas que cumplen con la condición
nueva_df = df[df['valor'] <= 5].groupby('id').apply(combinar_filas).reset_index(drop=True)

# Agregar las filas que no cumplen con la condición
# nueva_df = nueva_df.append(df[df['valor'] > 5], ignore_index=True)

print(nueva_df)


   id  valor   lista
0   1     10  [a, b]
1   1      4  [c, d]
2   2      8  [e, f]
3   2      2  [g, h]
4   3      6  [i, j]
   id   lista
0   1  [c, d]
1   2  [g, h]


C:\Users\alfre\AppData\Local\Temp\ipykernel_18644\3267949962.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nueva_df = df[df['valor'] <= 5].groupby('id').apply(combinar_filas).reset_index(drop=True)


In [29]:
import pandas as pd
import numpy as np

# Datos de ejemplo
data = {
    'InvoiceNo': [563852, 558623, 577551, 537026, 552884],
    'CustomerID': [12395, 12395, 12395, 12395, 12395],
    'InvoiceDate': ['2011-08-19 15:18:00', '2011-06-30 16:52:00', '2011-11-20 15:21:00', '2010-12-03 16:35:00', '2011-05-12 10:15:00'],
    'StockCode': [
        ['85066', '84816', '22457', '22326', '22352', '22329', 'POST'],
        ['22326', '22661', '21559', '21561', '23307', '22726', '22727'],
        ['23487', '23407', '23389', '23390', '21559', '84375', '22661'],
        ['84375', '21217', '21212', '21977', '22417', '21975', '21977'],
        ['22727', '22728', '22729', '22730', '22666', '84380', '22900']
    ]
}

# Crear DataFrame
df = pd.DataFrame(data)

# Convertir 'InvoiceDate' a datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Función para realizar el proceso de concatenación y mantener solo la entrada más reciente para cada CustomerID
def concat_and_keep_recent(group):
    # Encontrar la fecha más reciente
    max_date = group['InvoiceDate'].max()
    # Filtrar filas con la fecha más reciente
    recent_entry = group[group['InvoiceDate'] == max_date].copy()  # Usar .copy() para evitar SettingWithCopyWarning
    # Filtrar filas anteriores para concatenar las listas de valores
    previous_entries = group[group['InvoiceDate'] < max_date]
    # Concatenar las listas de valores utilizando np.concatenate()
    recent_entry['StockCode'] = [np.concatenate(previous_entries['StockCode'].values)] #* len(recent_entry)
    return recent_entry

# Aplicar la función a cada grupo de CustomerID
df_concatenated = df.groupby('CustomerID').apply(concat_and_keep_recent).reset_index(drop=True)
print(df_concatenated.InvoiceNo)
print(np.array(df_concatenated.StockCode))


0    577551
Name: InvoiceNo, dtype: int64
[array(['85066', '84816', '22457', '22326', '22352', '22329', 'POST',
        '22326', '22661', '21559', '21561', '23307', '22726', '22727',
        '84375', '21217', '21212', '21977', '22417', '21975', '21977',
        '22727', '22728', '22729', '22730', '22666', '84380', '22900'],
       dtype='<U5')                                                    ]


C:\Users\alfre\AppData\Local\Temp\ipykernel_18644\825962291.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_concatenated = df.groupby('CustomerID').apply(concat_and_keep_recent).reset_index(drop=True)


In [30]:
non_groupable_invoices_mask = ~df_invoice.isin(df_invoice_filtered_10min)

In [31]:
def concat_and_keep_recent(group):
    # Encontrar la fecha más reciente
    max_date = group['InvoiceDate'].max()
    # Filtrar filas con la fecha más reciente
    recent_entry = group[group['InvoiceDate'] == max_date].copy()  # Usar .copy() para evitar SettingWithCopyWarning
    # Filtrar filas anteriores para concatenar las listas de valores
    previous_entries = group[group['InvoiceDate'] < max_date]
    # Concatenar las listas de valores utilizando np.concatenate()
    recent_entry['StockCode'] = [np.concatenate(previous_entries['StockCode'].values, recent_entry['StockCode'].values)] #* len(recent_entry)
    return recent_entry


df_concatenated = df_invoice_filtered_10min.groupby('CustomerID').apply(concat_and_keep_recent).reset_index(drop=True)
# df_invoice_filtered_10min = concat_and_keep_recent(df_invoice_filtered_10min)
df_invoice_filtered_10min.head()

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
df_invoice_filtered.loc[df_invoice_filtered.loc[:, 'CustomerID'] == 13047, :]

,InvoiceNo,CustomerID,InvoiceDate,StockCode
2,536367,13047,2010-12-01 08:34:00,"[84879, 22745, 22748, 22749, 22310, 84969, 226..."
3,536368,13047,2010-12-01 08:34:00,"[22960, 22913, 22912, 22914]"
4,536369,13047,2010-12-01 08:35:00,[21756]
2800,543113,13047,2011-02-03 13:06:00,"[16161P, 22766, 22708, 23231, 85015, 85016, 22..."
5126,548331,13047,2011-03-30 13:28:00,"[22960, 22722, 22720, 20972, 22274, 84969, 226..."
6602,551846,13047,2011-05-04 14:11:00,"[23152, 23182, 23161, 23160, 23163, 23162, 231..."
9123,557299,13047,2011-06-19 13:56:00,"[23092, 23112, 23110, 23111, 23118, 23093, 230..."
11388,562437,13047,2011-08-04 17:58:00,"[22623, 16161P, 21754, 21755, 21901, 21900, 48..."
15263,571185,13047,2011-10-14 11:15:00,"[22739, 23435, 22960, 22722, 22720, 20972, 226..."
16998,575043,13047,2011-11-08 12:06:00,[M]


In [ ]:
df_unique_desc_code = (
    df_std
    .loc[:, ['StockCode', 'Description']]
    .dropna()
    .drop_duplicates()
)
df_unique_desc = (
    df_std
    .loc[:, 'Description']
    .dropna()
    .drop_duplicates()
)
df_unique_code = (
    df_std
    .loc[:, 'StockCode']
    .dropna()
    .drop_duplicates()
)
len(df_unique_desc_code), len(df_unique_desc), len(df_unique_code)

(4161, 4026, 3922)

In [ ]:
df_unique_no = (
    df
    .loc[:, 'InvoiceNo']
    .dropna()
    .drop_duplicates()
)

df_unique_date = (
    df
    .loc[:, 'InvoiceDate']
    .dropna()
    .drop_duplicates()
)
len(df_unique_date), len(df_unique_no)

(23260, 25900)

In [ ]:
import pandas as pd

# Suponiendo que tienes un DataFrame llamado df con las columnas transaction_id y product_id

# Ejemplo de DataFrame
data = {
    'transaction_id': [1, 1, 2, 2, 2, 3],
    'product_id': [101, 102, 201, 202, 203, 301]
}
df = pd.DataFrame(data)

# Agrupar por transaction_id y agregar product_id a una lista
new_df = df.groupby('transaction_id')['product_id'].apply(list).reset_index()

# Renombrar la columna resultante
new_df.columns = ['transaction_id', 'product_ids_array']

print(new_df)


   transaction_id product_ids_array
0               1        [101, 102]
1               2   [201, 202, 203]
2               3             [301]


In [ ]:
no_unique = df_std.loc[:, 'InvoiceNo'].unique().astype(str)
for no in no_unique:
    try:
        int(no)
    except:
        print('Letter here')
no_unique

Letter here


array(['536365', '536366', '536367', ..., '581585', '581586', '581587'],
      dtype='<U7')

In [ ]:
import numpy as np
import re

# Assuming arr is your numpy array containing strings

# Define a regular expression pattern to match strings starting with "c"
pattern = re.compile('b$', flags=re.IGNORECASE)

# Initialize a boolean mask indicating whether each element starts with "c"
starts_with_c_mask = np.array([bool(pattern.match(element)) for element in no_unique])

# Use the mask to filter elements starting with "c"
strings_starting_with_c = no_unique[starts_with_c_mask]

print(strings_starting_with_c)


[]


In [ ]:
df_unique_desc = (
    df
    .loc[:, 'Description']
    .drop_duplicates()
)
print(len(df_unique_desc))
df_unique_desc.head()

4224


0     WHITE HANGING HEART T-LIGHT HOLDER
1                    WHITE METAL LANTERN
2         CREAM CUPID HEARTS COAT HANGER
3    KNITTED UNION FLAG HOT WATER BOTTLE
4         RED WOOLLY HOTTIE WHITE HEART.
Name: Description, dtype: object

In [ ]:
import re
a = 'hola123mundo45.6'
num = re.findall(r'\d+', a)
print(num)

['123', '45', '6']


In [ ]:
import pandas as pd
import re

# Supongamos que tienes una serie llamada 'serie'
serie = df.loc[:, 'Description'].astype(str)

# Define una función para encontrar la longitud de los números en una cadena
def num_in_desc_len(desc):
    nums = re.findall(r'\d+', desc)  # Busca todos los números en la cadena
    len_num = sum(len(num) for num in nums)
    return len_num

# Aplica la función a cada elemento de la serie
longitudes_numeros = serie.apply(num_in_desc_len)

# Crea un DataFrame con la serie original y las longitudes de los números encontrados
df_nums = pd.DataFrame({'serie': serie, 'longitudes_numeros': longitudes_numeros})

# Imprime el DataFrame resultante
print(df_nums)


                                      serie  longitudes_numeros
0        WHITE HANGING HEART T-LIGHT HOLDER                   0
1                       WHITE METAL LANTERN                   0
2            CREAM CUPID HEARTS COAT HANGER                   0
3       KNITTED UNION FLAG HOT WATER BOTTLE                   0
4            RED WOOLLY HOTTIE WHITE HEART.                   0
...                                     ...                 ...
541904          PACK OF 20 SPACEBOY NAPKINS                   2
541905         CHILDREN'S APRON DOLLY GIRL                    0
541906        CHILDRENS CUTLERY DOLLY GIRL                    0
541907      CHILDRENS CUTLERY CIRCUS PARADE                   0
541908        BAKING SET 9 PIECE RETROSPOT                    1

[541909 rows x 2 columns]


In [ ]:
print(df_nums.loc[:, 'longitudes_numeros'].unique())
num_digits = 3
df_filtered = (
    df
    .loc[df_nums.loc[:, 'longitudes_numeros'] == num_digits, :]
    .loc[df.loc[:, 'Quantity'] > 0, :]
    .loc[df.loc[:, 'UnitPrice'] > 0, :]
)
df_filtered.tail(), len(df_filtered)

[0 1 2 3 4 5 8 6 7 9]


(       InvoiceNo StockCode                  Description  Quantity  \
 541496    581498     23354  6 GIFT TAGS 50'S CHRISTMAS          9   
 541612    581514     21705      BAG 500g SWIRLY MARBLES        84   
 541615    581516     21705      BAG 500g SWIRLY MARBLES        24   
 541690    581538     23319  BOX OF 6 MINI 50'S CRACKERS         1   
 541815    581579     23319  BOX OF 6 MINI 50'S CRACKERS        12   
 
                InvoiceDate  UnitPrice  CustomerID         Country  
 541496 2011-12-09 10:26:00       1.63       15287  United Kingdom  
 541612 2011-12-09 11:20:00       0.39       17754  United Kingdom  
 541615 2011-12-09 11:26:00       0.39       14422  United Kingdom  
 541690 2011-12-09 11:34:00       2.49       14446  United Kingdom  
 541815 2011-12-09 12:19:00       2.49       17581  United Kingdom  ,
 1973)

In [ ]:
import pandas as pd
import re

# Supongamos que tienes una serie llamada 'serie'
serie = df.loc[:, 'Description'].astype(str)

# Define una función para encontrar la longitud de los números en una cadena
def encontrar_longitud_numeros(cadena):
    numeros_encontrados = re.findall(r'\d+', cadena)  # Busca todos los números en la cadena
    longitudes = [len(numero) for numero in numeros_encontrados]  # Calcula la longitud de cada número
    return longitudes

# Aplica la función a cada elemento de la serie
longitudes_numeros = serie.apply(encontrar_longitud_numeros)

# Crea un DataFrame con la serie original y las longitudes de los números encontrados
df = pd.DataFrame({'serie': serie, 'longitudes_numeros': longitudes_numeros})

# Imprime el DataFrame resultante
print(df)


                                      serie longitudes_numeros
0        WHITE HANGING HEART T-LIGHT HOLDER                 []
1                       WHITE METAL LANTERN                 []
2            CREAM CUPID HEARTS COAT HANGER                 []
3       KNITTED UNION FLAG HOT WATER BOTTLE                 []
4            RED WOOLLY HOTTIE WHITE HEART.                 []
...                                     ...                ...
541904          PACK OF 20 SPACEBOY NAPKINS                [2]
541905         CHILDREN'S APRON DOLLY GIRL                  []
541906        CHILDRENS CUTLERY DOLLY GIRL                  []
541907      CHILDRENS CUTLERY CIRCUS PARADE                 []
541908        BAKING SET 9 PIECE RETROSPOT                 [1]

[541909 rows x 2 columns]


In [ ]:
df_unique_desc = (
    df
    .loc[:, 'Description']
    .dropna()
    .drop_duplicates()
)
len(df_unique_desc)

KeyError: 'Description'

In [ ]:
df_unique_desc.head()

0     WHITE HANGING HEART T-LIGHT HOLDER
1                    WHITE METAL LANTERN
2         CREAM CUPID HEARTS COAT HANGER
3    KNITTED UNION FLAG HOT WATER BOTTLE
4         RED WOOLLY HOTTIE WHITE HEART.
Name: Description, dtype: object

In [ ]:
df_unique_desc = df_unique_desc.loc.astype(str)
df_unique_desc['Description']

IndexingError: Too many indexers

In [ ]:
df_unique_desc[~df_unique_desc.str.contains('[a-z]') & df_unique_desc.str.contains('\d')]

TypeError: bad operand type for unary ~: 'float'

In [ ]:
df_unique_desc['Numero_Extraido'] = df_unique_desc.str.extract(r'(\d+)')
df_unique_desc.head()

0     WHITE HANGING HEART T-LIGHT HOLDER
1                    WHITE METAL LANTERN
2         CREAM CUPID HEARTS COAT HANGER
3    KNITTED UNION FLAG HOT WATER BOTTLE
4         RED WOOLLY HOTTIE WHITE HEART.
Name: Description, dtype: object

Podemos completar 

In [ ]:
df_unique_desc_code = (
    df
    .loc[:, ['StockCode', 'Description']]
    .dropna()
    .drop_duplicates()
)
len(df_unique_desc_code)

4792

In [ ]:
count_by_name = df.groupby('Description')['StockCode'].nunique()
count_by_name.sort_values(inplace=True)

In [ ]:
df.loc[df.loc[:, 'Description'] == 'wrongly marked 23343', :].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
415582,572547,20713,wrongly marked 23343,200,2011-10-24 17:01:00,0.0,15287,United Kingdom


In [ ]:
df.loc[df.loc[:, 'StockCode'] == 20713, :].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
522,536409,20713,JUMBO BAG OWLS,1,2010-12-01 11:45:00,1.95,17908,United Kingdom
1117,536527,20713,JUMBO BAG OWLS,10,2010-12-01 13:04:00,1.95,12662,Germany
1439,536542,20713,JUMBO BAG OWLS,30,2010-12-01 14:11:00,1.95,16456,United Kingdom
6381,536938,20713,JUMBO BAG OWLS,20,2010-12-03 12:05:00,1.95,14680,United Kingdom
7788,537054,20713,JUMBO BAG OWLS,3,2010-12-05 11:40:00,1.95,16931,United Kingdom


In [ ]:
count_by_name, count_by_name.index, count_by_name.values

(Description
 20713                           1
 POSSIBLE DAMAGES OR LOST?       1
 POSTAGE                         1
 POSTE FRANCE CUSHION COVER      1
 POSY CANDY BAG                  1
                              ... 
 found                          25
 damages                        43
 damaged                        43
 ?                              47
 check                         146
 Name: StockCode, Length: 4223, dtype: int64,
 Index([                             20713,        'POSSIBLE DAMAGES OR LOST?',
                                 'POSTAGE',       'POSTE FRANCE CUSHION COVER',
                          'POSY CANDY BAG', 'POTTERING IN THE SHED METAL SIGN',
                           'POTTERING MUG',   'POTTING SHED CANDLE CITRONELLA',
                'POTTING SHED ROSE CANDLE',      'POTTING SHED SEED ENVELOPES',
        ...
                             'thrown away',           'Unsaleable, destroyed.',
                                 'Damaged',                     

In [ ]:
df_unique_desc = (
    df
    .loc[:, 'Description']
    .dropna()
    .drop_duplicates()
)
df_unique_stockcode = (
    df
    .loc[:, 'StockCode']
    .dropna()
    .drop_duplicates()
)
df_unique_desc_code = (
    df
    .loc[:, ['Description', 'StockCode']]
    .dropna()
    .drop_duplicates()
)
len(df_unique_desc), len(df_unique_stockcode), len(df_unique_desc_code)

(4223, 4070, 4792)

In [ ]:
df_unique_desc.head()

0     WHITE HANGING HEART T-LIGHT HOLDER
1                    WHITE METAL LANTERN
2         CREAM CUPID HEARTS COAT HANGER
3    KNITTED UNION FLAG HOT WATER BOTTLE
4         RED WOOLLY HOTTIE WHITE HEART.
Name: Description, dtype: object

In [ ]:
df.loc[df.loc[:, 'Description'] == "WHITE HANGING HEART T-LIGHT HOLDER", :].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
49,536373,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,17850,United Kingdom
66,536375,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,17850,United Kingdom
220,536390,85123A,WHITE HANGING HEART T-LIGHT HOLDER,64,2010-12-01 10:19:00,2.55,17511,United Kingdom
262,536394,85123A,WHITE HANGING HEART T-LIGHT HOLDER,32,2010-12-01 10:39:00,2.55,13408,United Kingdom


In [ ]:
(
    df
    .describe()
    .drop('count')
    .T
)

,mean,min,25%,50%,75%,max,std
Quantity,9.55225,-80995.0,1.0,3.0,10.0,80995.0,218.081158
InvoiceDate,2011-07-04 13:34:57.156386048,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
UnitPrice,4.611114,-11062.06,1.25,2.08,4.13,38970.0,96.759853
CustomerID,15287.518434,12346.0,14367.0,15287.0,16255.0,18287.0,1484.746041


Nos fijamos que existen valores negativos tanto en cantidad (Quantity) como en precio de la unidad (UnitPrice).  
Veamos cuantas entradas tienen este problema para estudiar que hacer.

In [ ]:
df_negs = df.loc[(df.loc[:, 'Quantity'] <= 0) | (df.loc[:, 'UnitPrice'] <= 0), :]
df_negs.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom


In [ ]:
len(df_negs)

11805

Vemos que existen más de 11 mil entradas con valores negativos en alguna de las columnas "Quantity" y "UnitPrice".  

In [ ]:
negs_ratio = round(len(df_negs)/len(df), 2)
print(negs_ratio)

0.02


Vemos que solo el 2% de los datos contienen valores negativos, consideramos que es una parte pequeña de los datos y por tanto los eliminamos del dataset.

In [ ]:
df_std = (
    pd.concat([df, df_negs])
    .drop_duplicates(keep=False)
)
df_std.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


Comprobamos que hemos realizado correctamente la transformación.

In [ ]:
df_negs = df_std.loc[(df_std.loc[:, 'Quantity'] <= 0) | (df_std.loc[:, 'UnitPrice'] <= 0), :]
len(df_negs)

0

In [ ]:
df.loc[df.loc[:, 'Description'].isna(), :].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,15287,United Kingdom
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,15287,United Kingdom
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,15287,United Kingdom
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,15287,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,15287,United Kingdom


In [ ]:
df.loc[:, 'StockCode'].nunique(), df.loc[:, 'Description'].nunique()

(4070, 4223)

In [ ]:
num_uniques = len(df.loc[:, ['StockCode', 'Description']].drop_duplicates(keep=False))
print(f'El número de productos únicos es {num_uniques}')

El número de productos únicos es 1429


In [ ]:
import pandas as pd

# Supongamos que df es tu DataFrame
df1 = pd.DataFrame({
    'columna1': ['a', 'c', 'a', 'c', 'c'],
    'columna2': ['x', 'z', 'x', 'z', 'z']
})

# Eliminar filas duplicadas y obtener un nuevo DataFrame con filas únicas
df_filas_unicas = df1.drop_duplicates()

print("DataFrame original:")
print(df1)

print("\nDataFrame con filas únicas:")
print(df_filas_unicas)

print("\nDataFrame keep=False:")
print(df1.drop_duplicates(keep=False))


DataFrame original:
  columna1 columna2
0        a        x
1        c        z
2        a        x
3        c        z
4        c        z

DataFrame con filas únicas:
  columna1 columna2
0        a        x
1        c        z

DataFrame keep=False:
Empty DataFrame
Columns: [columna1, columna2]
Index: []
